In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# 기본 설정
base_model_id = "Qwen/Qwen2.5-Coder-7B-Instruct"  # 기본 모델 경로 또는 HF 모델명
adapter_path = "./Qwen2.5-all-en/checkpoint-128481"      # LoRA 어댑터 경로
merged_path = "./merged_qwen2.5-all-en"       # 저장할 경로

# 베이스 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    low_cpu_mem_usage=True,
    torch_dtype="auto"
)

# PEFT 모델 로드 및 병합
peft_model = PeftModel.from_pretrained(model, adapter_path)
merged_model = peft_model.merge_and_unload()

# 병합된 모델 저장
merged_model.save_pretrained(
    merged_path,
    safe_serialization=True,
    max_shard_size="5GB"
)

# 토크나이저 저장
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.save_pretrained(merged_path)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  4.72it/s]


('/root/workspace/text_to_sql/merged_qwen2.5-all-en/tokenizer_config.json',
 '/root/workspace/text_to_sql/merged_qwen2.5-all-en/special_tokens_map.json',
 '/root/workspace/text_to_sql/merged_qwen2.5-all-en/vocab.json',
 '/root/workspace/text_to_sql/merged_qwen2.5-all-en/merges.txt',
 '/root/workspace/text_to_sql/merged_qwen2.5-all-en/added_tokens.json',
 '/root/workspace/text_to_sql/merged_qwen2.5-all-en/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# (1) 병합된 모델 다시 로드
merged_model = AutoModelForCausalLM.from_pretrained(merged_path)
tokenizer = AutoTokenizer.from_pretrained(merged_path)

# (2) push_to_hub
merged_model.push_to_hub("daje/Qwen2.5-coder-7B-en-all-merged")   # 원하는 레포 이름
tokenizer.push_to_hub("daje/Qwen2.5-coder-7B-en-all-merged")